Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [16]:
# neural network with L2
batch_size = 128
hidden_nodes_1 = 1024
beta=0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_1]))
  biases = tf.Variable(tf.zeros([hidden_nodes_1]))

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_out_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)  
    
  logits = tf.matmul(hidden_out_1, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Loss function using L2 Regularization
  regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(loss + beta * regularizer)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_2) + biases_2)
  test_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)  
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden_1, weights_2) + biases_2)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

## without l2 regularization
# Minibatch loss at step 3000: 12.300850
# Minibatch accuracy: 75.0%
# Validation accuracy: 79.2%
# Test accuracy: 86.1%

Initialized
Minibatch loss at step 0: 3502.820068
Minibatch accuracy: 7.0%
Validation accuracy: 35.7%
Minibatch loss at step 500: 21.254763
Minibatch accuracy: 86.7%
Validation accuracy: 84.6%
Minibatch loss at step 1000: 0.809048
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.800774
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.753754
Minibatch accuracy: 82.8%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.621184
Minibatch accuracy: 85.2%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.715804
Minibatch accuracy: 86.7%
Validation accuracy: 83.5%
Test accuracy: 90.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [19]:
num_steps = 3001

train_dataset_small = train_dataset[:500, :]
train_labels_small = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3481.736572
Minibatch accuracy: 2.3%
Validation accuracy: 27.9%
Minibatch loss at step 500: 21.042072
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 0.452020
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 0.286921
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 0.279740
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.277642
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.278078
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Test accuracy: 85.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
# neural network with L2 and dropout
batch_size = 128
hidden_nodes_1 = 1024
beta=0.01
dropout_keep_prob=0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_1]))
  biases = tf.Variable(tf.zeros([hidden_nodes_1]))

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h_out_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)  
  h_out_1_drop = tf.nn.dropout(h_out_1, dropout_keep_prob)
    
  logits = tf.matmul(h_out_1_drop, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Loss function using L2 Regularization
  regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(loss + beta * regularizer)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_2) + biases_2)
  test_hidden_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)  
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden_1, weights_2) + biases_2)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

## without dropout
# Minibatch loss at step 3000: 0.715804
# Minibatch accuracy: 86.7%
# Validation accuracy: 83.5%
# Test accuracy: 90.1%

Initialized
Minibatch loss at step 0: 3537.391357
Minibatch accuracy: 9.4%
Validation accuracy: 42.0%
Minibatch loss at step 500: 21.298672
Minibatch accuracy: 86.7%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.805540
Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.793186
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.762596
Minibatch accuracy: 82.0%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.629268
Minibatch accuracy: 85.2%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.722478
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Test accuracy: 90.0%


In [21]:
num_steps = 3001

train_dataset_small = train_dataset[:500, :]
train_labels_small = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3428.678467
Minibatch accuracy: 10.2%
Validation accuracy: 25.3%
Minibatch loss at step 500: 21.053333
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 0.454433
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 1500: 0.288190
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 2000: 0.281060
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 0.278630
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.278895
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 84.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
# neural network with L2, dropout and learning rate decay
batch_size = 128

h_layer_1 = 1024
h_layer_2 = 300
h_layer_3 = 50
beta=0.0001


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h_layer_1], stddev=np.sqrt(2.0/784)))
  biases_1 = tf.Variable(tf.zeros([h_layer_1]))

  weights_2 = tf.Variable(
    tf.truncated_normal([h_layer_1, h_layer_2], stddev=np.sqrt(2.0/h_layer_1)))
  biases_2 = tf.Variable(tf.zeros([h_layer_2]))
    
  weights_3 = tf.Variable(
    tf.truncated_normal([h_layer_2, h_layer_3], stddev=np.sqrt(2.0/h_layer_2)))
  biases_3 = tf.Variable(tf.zeros([h_layer_3]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([h_layer_3, num_labels], stddev=np.sqrt(2.0/h_layer_3)))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), 0.5)
  logits_2 = tf.nn.dropout(tf.nn.relu(tf.matmul(logits_1, weights_2) + biases_2), 0.7)
  logits_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(logits_2, weights_3) + biases_3), 0.8)
  logits_out = tf.matmul(logits_3, weights_out) + biases_out

  
  
  # Loss function using L2 Regularization
  regularizer = beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + 
                      tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_out))
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits_out, tf_train_labels)) + regularizer

  # Optimizer.
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken
  learning_rate = tf.train.exponential_decay(0.4, global_step, 3000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_out)

  # validation
  valid_logits_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_logits_2 = tf.nn.relu(tf.matmul(valid_logits_1, weights_2) + biases_2)
  valid_logits_3 = tf.nn.relu(tf.matmul(valid_logits_2, weights_3) + biases_3)
  valid_logits_out = tf.matmul(valid_logits_3, weights_out) + biases_out
  valid_prediction = tf.nn.softmax(valid_logits_out)
    
  # test
  test_logits_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_logits_2 = tf.nn.relu(tf.matmul(test_logits_1, weights_2) + biases_2)
  test_logits_3 = tf.nn.relu(tf.matmul(test_logits_2, weights_3) + biases_3)
  test_logits_out = tf.matmul(test_logits_3, weights_out) + biases_out
  test_prediction = tf.nn.softmax(test_logits_out)  

In [6]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  #tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

## with L2 dropout
# Minibatch loss at step 3000: 0.715804
# Minibatch accuracy: 86.7%
# Validation accuracy: 83.5%
# Test accuracy: 90.1%

## with L2, dropout, learning_rate_decay


## with L2, dropout, learning_rate_decay, 3 layers

Initialized
Minibatch loss at step 0: 2.838426
Minibatch accuracy: 10.2%
Validation accuracy: 26.0%
Minibatch loss at step 500: 0.607321
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.582626
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Minibatch loss at step 1500: 0.684422
Minibatch accuracy: 82.8%
Validation accuracy: 86.8%
Minibatch loss at step 2000: 0.607610
Minibatch accuracy: 85.2%
Validation accuracy: 87.1%
Minibatch loss at step 2500: 0.502350
Minibatch accuracy: 86.7%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.649264
Minibatch accuracy: 83.6%
Validation accuracy: 88.0%
Minibatch loss at step 3500: 0.545224
Minibatch accuracy: 86.7%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 0.491443
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 4500: 0.507077
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 5000: 0.463246
Minibatch accuracy: 89.8%
Validation accurac